In [1]:
# import sys
# !{sys.executable} -m pip install python_dotenv
# !{sys.executable} -m pip install python_binance
import os
from binance.client import Client
from binance.spot import Spot

import pandas as pd
from dotenv import load_dotenv
pd.set_option("display.precision", 8)


In [2]:
# read from local .env file
load_dotenv() 

api_key = os.environ['BINANCE_API_KEY']
api_secret = os.environ['BINANCE_API_SECRET']

client = Client()
spotClient = Spot()
# myClient = Client(api_key, api_secret, testnet=True)

In [3]:
# INPUT
## SET this to get d2, d3, d4
day_aggregation = 10

## ema (20 or 25)
ema_size = 25

## A symbol with streak >= this will be printed out
streak_threshold = 20

## how many day to query
quantity = 6000

quote_asset = "BTC"


In [4]:
# TIMEFRAME
## set timeframe to query binance, based on day_aggregation
do_manual_aggregation = True
timeframe = '1d'
if day_aggregation == 3:
    timeframe = '3d'
    do_manual_aggregation = False
elif day_aggregation == 7:
    timeframe = '1w'
    do_manual_aggregation = False
elif day_aggregation == 30:
    timeframe = '1M'
    do_manual_aggregation = False

In [5]:
# FUNCTIONS
## Function to query Binance for candlestick data
def get_candlestick_data(symbol, timeframe, qty):
    # Retrieve the raw data
    raw_data = spotClient.klines(symbol=symbol, interval=timeframe, limit=qty)
    # Set up the return array
    converted_data = []
    # Convert each element into a Python dictionary object, then add to converted_data
    for candle in raw_data:
        # Dictionary object
        converted_candle = {
            'time': candle[0],
            'open': float(candle[1]),
            'high': float(candle[2]),
            'low': float(candle[3]),
            'close': float(candle[4]),
            'volume': float(candle[5]),
            'close_time': candle[6],
            'quote_asset_volume': float(candle[7]),
            'number_of_trades': int(candle[8]),
            'taker_buy_base_asset_volume': float(candle[9]),
            'taker_buy_quote_asset_volume': float(candle[10])
        }
        # Add to converted_data
        converted_data.append(converted_candle)
    # Return converted data
    return converted_data

## Define function to calculate an arbitrary EMA      
def calc_generic_ema(dataframe, symbol, timeframe, ema_size):
    # Create column string
    ema_name = "ema_" + str(ema_size)
    # Create the multiplier
    multiplier = 2/(ema_size + 1)
    # Calculate the initial value (SMA)
    # pd.set_option('display.max_columns', None) # <- use this to show all columns
    # pd.set_option('display.max_rows', None) # <- use this to show all the rows
    initial_mean = dataframe['close'].head(ema_size).mean()

    # Iterate through Dataframe
    for i in range(len(dataframe)):
        if i == ema_size:
            dataframe.loc[i, ema_name] = initial_mean
        elif i > ema_size:
            ema_value = dataframe.loc[i, 'close'] * multiplier + dataframe.loc[i-1, ema_name]*(1-multiplier)
            dataframe.loc[i, ema_name] = ema_value
        else:
            dataframe.loc[i, ema_name] = 0.00
    # print(dataframe) # <- use this to print the dataframe if you want to inspect
    return dataframe

In [6]:
# GET SYMBOL LIST
info = client.get_exchange_info()
coinlist = []
for c in info['symbols']:
    if c['quoteAsset']==quote_asset and c['status']=="TRADING":
        coinlist.append(c['symbol'])
        
print(len(coinlist))
# sublist = coinlist[0:2]
sublist = coinlist
print('Scanning ' + str(len(sublist)) + ' trading pairs...')

283
Scanning 2 trading pairs...


In [7]:
# PROGRESS BAR
from ipywidgets import IntProgress
from IPython.display import display
import time

max_count = len(sublist)

f = IntProgress(min=0, max=max_count) # instantiate the bar
display(f) # display the bar

count = 0

# MAIN
for idx, symbol in enumerate(sublist):
    # update progress bar
    f.value += 1
    
    raw_data = get_candlestick_data(symbol=symbol, timeframe=timeframe, qty=1000)
    ## Convert into Dataframe
    df = pd.DataFrame(raw_data)

    if do_manual_aggregation:
        df['d_index'] = df.index//day_aggregation
        # get high / low
        df['d_high'] = df.groupby('d_index')['high'].transform(max)
        df['d_low'] = df.groupby(['d_index'])['low'].transform(min)

        # get open / close

        df['d_open'] = df.groupby(['d_index'])['open'].transform('first')
        df['d_close'] = df.groupby(['d_index'])['close'].transform('last')

        df = df.groupby('d_index').first()

        # clean up d1 column
        del df['open']
        del df['high']
        del df['low']
        del df['close']

        # rename d_column
        df = df.rename(columns={"d_high": "high", "d_low": "low", "d_close": "close", "d_open": "open" })

    df['time'] = pd.to_datetime(df['time'], unit='ms')
    # ema
    df = calc_generic_ema(df, symbol, timeframe, ema_size)

    # clean up

    del df['volume']
    del df['close_time']
    del df['quote_asset_volume']
    del df['number_of_trades']
    del df['taker_buy_base_asset_volume']
    del df['taker_buy_quote_asset_volume']

    # Calculate oversell
    df['oversell_day'] = 0
    df.loc[df['high'] < df['ema_25'], 'oversell_day'] = 1
    df.loc[df['ema_25'] == 0, 'oversell_day'] = 0


    df['streak'] = df.groupby((df["oversell_day"] != df["oversell_day"].shift()).cumsum()).cumcount()+1
    df.loc[df['oversell_day'] == 0, 'streak'] = 0
    
    if (df['streak'].iloc[-1] > streak_threshold):
        print('Symbol: '+ symbol + ' is having an oversell streak')
        display(df['streak'].iloc[-1])
#     display(df.tail(30))
        
    if f.value == max_count:
        print('Scan Completed!')

IntProgress(value=0, max=2)

Scan Completed!
